In [1]:
from qoop.core import state, ansatz

<img src = './docs/1.png' height ='800px'>

In [ ]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function
def V():
    return # choose an ansatz from qoop.core.ansatz

def Delta(rho):
    # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
    # see Eq. 1 Ref. [1]
    return # rho'

# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

<img src = './docs/2.png' height ='800px'>